In [2]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
df_str_rel = pd.read_csv('C:\\Users\\ADMIN\\Downloads\\tel_train.csv')
df_str_rel.head()

,PairID,Text,Score
0,TEL-train-00001,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...",0.88
1,TEL-train-00002,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...",0.87
2,TEL-train-00003,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...",0.87
3,TEL-train-00004,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...",0.84
4,TEL-train-00005,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...",0.84


In [4]:
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

,PairID,Text,Score,Split_Text
0,TEL-train-00001,"""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గురు...",0.88,"[""జమ్మూకాశ్మీర్లో మంచుకొండచరియలు విరిగిపడి గుర..."
1,TEL-train-00002,"""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన ...",0.87,"[""కూతురుపైనే కన్నతండ్రి అత్యాచారానికి పాల్పడిన..."
2,TEL-train-00003,"""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకుర...",0.87,"[""గంగా నదిని కాపాడేందుకు ప్రత్యేక చట్టం తీసుకు..."
3,TEL-train-00004,"""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేకం...",0.84,"[""కరోనా వైరస్పై అమెరికా ప్రభుత్వానికి వ్యతిరేక..."
4,TEL-train-00005,"""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు జ...",0.84,"[""కరోనా వైరస్ ( కొవిడ్ - 19 ) మనుషులపైనే కాదు ..."


In [5]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Specify the model name or path
model_name_or_path = "ai4bharat/indic-bert"

# Load the IndicBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path)

# Function to encode text and get embeddings
def encode_text(text):
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Calculate cosine similarity between two sentence embeddings
def calculate_similarity(sentence1, sentence2):
    embeddings1 = encode_text(sentence1)
    embeddings2 = encode_text(sentence2)
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]

sentence1 = "This is the first sentence."
sentence2 = "This is the second sentence."

similarity = calculate_similarity(sentence1, sentence2)
print(f"Cosine Similarity: {similarity:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Cosine Similarity: 0.9665


In [6]:
true_scores = df_str_rel['Score'].values
pred_scores = []

for index,row in df_str_rel.iterrows():
  s1,s2 = row["Text"].split("\n")

  # Overlap score
  pred_scores.append(calculate_similarity(s1,s2))

In [7]:
print("Spearman Correlation:", round(spearmanr(true_scores,pred_scores)[0],2))

Spearman Correlation: 0.45
